In [49]:
import json
import pandas as pd
pd.set_option("display.max_columns", 100)
import numpy as np

In [140]:
import hrvanalysis

In [142]:
print(hrvanalysis.__version__)

1.0.3


In [3]:
path = "/home/zozi/Documents/Code/Projects/Epilepsy_Project/Aura/res-v0_4/dev/01_tcp_ar/002/00000258/s002_2003_07_21/00000258_s002_t000.json"

features = "xqrs"

In [33]:
data = json.load(open(path, "r"))
df = pd.DataFrame(columns = list(data[features]), 
                    data = list(map(list, zip(*list(data[features].values())))))

In [22]:
data['infos']

{'sampling_freq': 400,
 'start_datetime': '2003-07-21T17:12:54',
 'exam_duration': 20,
 'ref_file': '00000258_s002_t000.edf'}

In [37]:
df

,qrs,rr_intervals,hr
0,124,377.5,158.0
1,275,665.0,90.0
2,541,667.5,89.0
3,808,672.5,89.0
4,1077,665.0,90.0
5,1343,670.0,89.0
6,1611,667.5,89.0
7,1878,672.5,89.0
8,2147,665.0,90.0
9,2413,670.0,89.0


In [38]:
df.rr_intervals.sum()

19620.0

In [34]:
len(df.rr_intervals.tolist())

30

In [24]:
df.rr_intervals.max

<bound method Series.max of 0     377.5
1     665.0
2     667.5
3     672.5
4     665.0
5     670.0
6     667.5
7     672.5
8     665.0
9     670.0
10    662.5
11    665.0
12    662.5
13    665.0
14    665.0
15    662.5
16    662.5
17    662.5
18    662.5
19    660.0
20    660.0
21    650.0
22    657.5
23    657.5
24    662.5
25    662.5
26    660.0
27    662.5
28    662.5
29    662.5
Name: rr_intervals, dtype: float64>

In [137]:
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values

# rr_intervals_list contains integer values of RR-interval
rr_intervals_list = df['rr_intervals']

# This remove outliers from signal
rr_intervals_without_outliers = remove_outliers(rr_intervals=rr_intervals_list,  
                                                low_rri=600, high_rri=670)
# This replace outliers nan values with linear interpolation
interpolated_rr_intervals = interpolate_nan_values(rr_intervals=rr_intervals_without_outliers,
                                                   interpolation_method="linear",                       
                                                   limit=10)#,
                                                   #limit_area=None,
                                                   #limit_direction='both')

# This remove ectopic beats from signal
nn_intervals_list = remove_ectopic_beats(rr_intervals=interpolated_rr_intervals, method="malik")
# This replace ectopic beats nan values with linear interpolation
interpolated_nn_intervals = interpolate_nan_values(rr_intervals=nn_intervals_list)

3 outlier(s) have been deleted.
The outlier(s) value(s) are : [377.5, 672.5, 672.5]
1 ectopic beat(s) have been deleted with malik rule.


In [138]:
interpolated_nn_intervals

[nan,
 nan,
 667.5,
 666.25,
 665.0,
 670.0,
 667.5,
 666.25,
 665.0,
 670.0,
 662.5,
 665.0,
 662.5,
 665.0,
 665.0,
 662.5,
 662.5,
 662.5,
 662.5,
 660.0,
 660.0,
 650.0,
 657.5,
 657.5,
 662.5,
 662.5,
 660.0,
 662.5,
 662.5,
 662.5]

In [117]:
s = pd.Series([0, 1, np.nan, 3])
s

0    0.0
1    1.0
2    NaN
3    3.0
dtype: float64

In [118]:
rr_intervals = df['rr_intervals']
low_rri=600
high_rri=700

In [120]:
rr_intervals_cleaned = [rri if high_rri >= rri >= low_rri else np.nan for rri in rr_intervals]
rr_intervals_cleaned

[nan,
 665.0,
 667.5,
 672.5,
 665.0,
 670.0,
 667.5,
 672.5,
 665.0,
 670.0,
 662.5,
 665.0,
 662.5,
 665.0,
 665.0,
 662.5,
 662.5,
 662.5,
 662.5,
 660.0,
 660.0,
 650.0,
 657.5,
 657.5,
 662.5,
 662.5,
 660.0,
 662.5,
 662.5,
 662.5]

In [125]:
series_rr_intervals_cleaned = pd.Series(rr_intervals_cleaned)
# Interpolate nan values and convert pandas object to list of values
series_rr_intervals_cleaned.interpolate(method='linear',
                                                                    #limit=None,
                                                                    #limit_area=None,
                                                                    limit_direction='both')

0     665.0
1     665.0
2     667.5
3     672.5
4     665.0
5     670.0
6     667.5
7     672.5
8     665.0
9     670.0
10    662.5
11    665.0
12    662.5
13    665.0
14    665.0
15    662.5
16    662.5
17    662.5
18    662.5
19    660.0
20    660.0
21    650.0
22    657.5
23    657.5
24    662.5
25    662.5
26    660.0
27    662.5
28    662.5
29    662.5
dtype: float64

In [111]:
interpolated_nn_intervals

[nan,
 nan,
 667.5,
 672.5,
 665.0,
 670.0,
 667.5,
 672.5,
 665.0,
 670.0,
 662.5,
 665.0,
 662.5,
 665.0,
 665.0,
 662.5,
 662.5,
 662.5,
 662.5,
 660.0,
 660.0,
 650.0,
 657.5,
 657.5,
 662.5,
 662.5,
 660.0,
 662.5,
 662.5,
 662.5]

In [114]:
df['rr_nan'] = rr_intervals_without_outliers
df.rr_nan = df.rr_intervals.interpolate(axis=0)
df

,qrs,rr_intervals,hr,rr_nan
0,124,377.5,158.0,377.5
1,275,665.0,90.0,665.0
2,541,667.5,89.0,667.5
3,808,672.5,89.0,672.5
4,1077,665.0,90.0,665.0
5,1343,670.0,89.0,670.0
6,1611,667.5,89.0,667.5
7,1878,672.5,89.0,672.5
8,2147,665.0,90.0,665.0
9,2413,670.0,89.0,670.0


In [97]:
series_rr_intervals_cleaned = pd.Series(rr_intervals_without_outliers)
# Interpolate nan values and convert pandas object to list of values
series_rr_intervals_cleaned.interpolate(method='linear', limit_area=None)


0       NaN
1     665.0
2     667.5
3     672.5
4     665.0
5     670.0
6     667.5
7     672.5
8     665.0
9     670.0
10    662.5
11    665.0
12    662.5
13    665.0
14    665.0
15    662.5
16    662.5
17    662.5
18    662.5
19    660.0
20    660.0
21    650.0
22    657.5
23    657.5
24    662.5
25    662.5
26    660.0
27    662.5
28    662.5
29    662.5
dtype: float64

In [45]:
from hrvanalysis import get_sampen, get_csi_cvi_features

In [64]:
sampen_features = get_sampen(nn_intervals_list)
sampen_features

{'sampen': 0.9808292530117262}

In [65]:
csi_cvi_features = get_csi_cvi_features(nn_intervals_list)
csi_cvi_features

{'csi': 1.8886649765339554,
 'cvi': 2.462436575543433,
 'Modified_csi': 44.20289136458043}

In [61]:
nn_intervals_list = nn_intervals_list[2:]
nn_intervals_list

[667.5,
 672.5,
 665.0,
 670.0,
 667.5,
 672.5,
 665.0,
 670.0,
 662.5,
 665.0,
 662.5,
 665.0,
 665.0,
 662.5,
 662.5,
 662.5,
 662.5,
 660.0,
 660.0,
 650.0,
 657.5,
 657.5,
 662.5,
 662.5,
 660.0,
 662.5,
 662.5,
 662.5]

In [67]:
diff_nn_intervals = np.diff(nn_intervals_list)
# measures the width of poincare cloud
sd1 = np.sqrt(np.std(diff_nn_intervals, ddof=1) ** 2 * 0.5)
# measures the length of the poincare cloud
sd2 = np.sqrt(2 * np.std(nn_intervals_list, ddof=1) ** 2 - 0.5 * np.std(diff_nn_intervals, ddof=1) ** 2)
ratio_sd2_sd1 = sd2 / sd1

In [68]:
diff_nn_intervals

array([  nan,   nan,   5. ,  -7.5,   5. ,  -2.5,   5. ,  -7.5,   5. ,
        -7.5,   2.5,  -2.5,   2.5,   0. ,  -2.5,   0. ,   0. ,   0. ,
        -2.5,   0. , -10. ,   7.5,   0. ,   5. ,   0. ,  -2.5,   2.5,
         0. ,   0. ])

In [69]:
sd1

nan

In [56]:
interpolated_nn_intervals

[nan,
 nan,
 667.5,
 672.5,
 665.0,
 670.0,
 667.5,
 672.5,
 665.0,
 670.0,
 662.5,
 665.0,
 662.5,
 665.0,
 665.0,
 662.5,
 662.5,
 662.5,
 662.5,
 660.0,
 660.0,
 650.0,
 657.5,
 657.5,
 662.5,
 662.5,
 660.0,
 662.5,
 662.5,
 662.5]

In [57]:
rr_intervals_without_outliers

[nan,
 665.0,
 667.5,
 672.5,
 665.0,
 670.0,
 667.5,
 672.5,
 665.0,
 670.0,
 662.5,
 665.0,
 662.5,
 665.0,
 665.0,
 662.5,
 662.5,
 662.5,
 662.5,
 660.0,
 660.0,
 650.0,
 657.5,
 657.5,
 662.5,
 662.5,
 660.0,
 662.5,
 662.5,
 662.5]

In [25]:
path_features = "/home/zozi/Documents/Code/Projects/Epilepsy_Project/Aura/feats-v0_4/dev/01_tcp_ar/002/00000258/s002_2003_07_21/00000258_s002_t000-xqrs.json"

In [26]:
def load_data(path):
    data = json.load(open(path, "r"))
    df = pd.DataFrame(columns = data['keys'], data = data['features'])
    return df

In [35]:
df_features = load_data(path_features)

In [36]:
df_features

,interval_index,interval_start_time,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
0,0.0,0.0,637.5,80.932070,50.407776,2.0,13.333333,2.0,13.333333,54.363197,665.0,292.5,0.085276,0.126952,96.289826,158.940397,89.552239,17.845787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1.0,10000.0,661.0,2.070197,1.336306,0.0,0.000000,0.0,0.000000,1.336306,662.5,5.0,0.002022,0.003132,90.772391,91.254753,90.566038,0.275356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2.0,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
